# Chapter 12.11: Multilingual learning

Load libraries:

In [ ]:
using JudiLing, DataFrames, Plots

### Loading data

In [ ]:
dat_multi = JudiLing.load_dataset("../dat/dat_multi.csv", delim="\t");

In [ ]:
S, words = JudiLing.load_S_matrix("../dat/S_multi.csv", header=true, sep=",");

### Monolingual learning

In [ ]:
dat_en = dat_multi[dat_multi.Language .== "ENG",:];
dat_de = dat_multi[dat_multi.Language .== "GER",:];
dat_zh = dat_multi[dat_multi.Language .== "MAN",:];

In [ ]:
cue_obj_en = JudiLing.make_cue_matrix(dat_en, grams=3, target_col=:Phon);
cue_obj_de = JudiLing.make_cue_matrix(dat_de, grams=3, target_col=:Phon);
cue_obj_zh = JudiLing.make_cue_matrix(dat_zh, grams=3, target_col=:Phon);

In [ ]:
S_en = S[1:190,:];
S_de = S[191:380,:];
S_zh = S[381:570,:];

In [ ]:
first(dat_en, 3)

#### Comprehension

In [ ]:
learn_seq = JudiLing.make_learn_seq(dat_en.simFreq, random_seed=314);
eval_size = 2747
n_evals = Int64(length(learn_seq)/eval_size); # 6
F = zeros(size(cue_obj_en.C)[2], size(S)[2]);

accs_comp = []
for i in 1:n_evals
    learn_seq_epoch = learn_seq[(((i-1)*eval_size)+1):i*eval_size]
    F = JudiLing.wh_learn(cue_obj_en.C, S_en, learn_seq=learn_seq_epoch, eta=0.01, weights=F)
    Shat = cue_obj_en.C * F
    acc = JudiLing.eval_SC(Shat, S)
    push!(accs_comp, acc)
end

In [ ]:
# making all the previous steps into a function
function monolingual_learning_comp(data, cue_obj, S, eval_size, whole_epoch_rep)
    learn_seq = repeat(JudiLing.make_learn_seq(data.simFreq, random_seed=314), whole_epoch_rep);
    evals = collect(Int64, 1:(length(learn_seq)/eval_size))
    F = zeros(size(cue_obj.C)[2], size(S)[2])

    accs = []
    for i in evals
        learn_seq_epoch = learn_seq[(((i-1)*eval_size)+1):i*eval_size]
        F = JudiLing.wh_learn(cue_obj.C, S, learn_seq=learn_seq_epoch, eta=0.01, weights=F, n_epochs=1)
        Shat = cue_obj.C * F
        acc = JudiLing.eval_SC(Shat, S)
        push!(accs, acc)
    end
    accs
end

In [ ]:
accs_en = monolingual_learning_comp(dat_en, cue_obj_en, S_en, 2747, 1);
accs_de = monolingual_learning_comp(dat_de, cue_obj_de, S_de, 2747, 1);
accs_zh = monolingual_learning_comp(dat_zh, cue_obj_zh, S_zh, 2747, 1);

In [ ]:
plot(accs_en, xlab="evaluation step", ylab="accuracy", label="English")
plot!(accs_de, label="German")
plot!(accs_zh, label="Mandarin")

In [ ]:
# double the learning: changing "whole_epoch_rep" from 1 to 2 
accs_en = monolingual_learning_comp(dat_en, cue_obj_en, S_en, 2747, 2);
accs_de = monolingual_learning_comp(dat_de, cue_obj_de, S_de, 2747, 2);
accs_zh = monolingual_learning_comp(dat_zh, cue_obj_zh, S_zh, 2747, 2);

In [ ]:
plot(accs_en, xlab="evaluation step", ylab="accuracy", label="English")
plot!(accs_de, label="German")
plot!(accs_zh, label="Mandarin")

#### Production

In [ ]:
learn_seq = JudiLing.make_learn_seq(dat_en.simFreq, random_seed=314);
eval_size = 2747
n_evals = Int64(length(learn_seq)/eval_size); # 6
F = zeros(size(cue_obj_en.C)[2], size(S)[2])
G = zeros(size(S)[2], size(cue_obj_en.C)[2])

accs_prod = []
for i in 1:n_evals
    learn_seq_epoch = learn_seq[(((i-1)*eval_size)+1):i*eval_size]
    F = JudiLing.wh_learn(cue_obj_en.C, S_en, learn_seq=learn_seq_epoch, eta=0.01, weights=F)
    G = JudiLing.wh_learn(S_en, cue_obj_en.C, learn_seq=learn_seq_epoch, eta=0.01, weights=G)
    Chat = S * G

    #learn path
    res = JudiLing.learn_paths(dat_en, cue_obj_en, S_en, F, Chat, threshold=0.01, verbose=false)
    acc_p = JudiLing.eval_acc(res, cue_obj_en)
    push!(accs_prod, acc_p)
end

In [ ]:
accs_prod

In [ ]:
function monolingual_learning_prod(data, cue_obj, S, eval_size, whole_epoch_rep)

    learn_seq = repeat(JudiLing.make_learn_seq(Int64.(data.simFreq), random_seed=314), whole_epoch_rep);
    evals = collect(Int64, 1:(length(learn_seq)/eval_size))
    F = zeros(size(cue_obj.C)[2], size(S)[2])
    G = zeros(size(S)[2], size(cue_obj.C)[2])

    accs_prod = []
    for i in evals
        learn_seq_epoch = learn_seq[(((i-1)*eval_size)+1):i*eval_size]
        F = JudiLing.wh_learn(cue_obj.C, S, learn_seq=learn_seq_epoch, eta=0.01, weights=F, n_epochs=1)
        G = JudiLing.wh_learn(S, cue_obj.C, learn_seq=learn_seq_epoch, eta=0.01, weights=G, n_epochs=1)
        Chat = S * G

        #learn path
        res = JudiLing.learn_paths(data, cue_obj, S, F, Chat, threshold=0.01, verbose=false)
        acc_p = JudiLing.eval_acc(res, cue_obj)
        push!(accs_prod, acc_p)
    end
    accs_prod
end

In [ ]:
prod_en = monolingual_learning_prod(dat_en, cue_obj_en, S_en, 2747, 1);
prod_de = monolingual_learning_prod(dat_de, cue_obj_de, S_de, 2747, 1);
prod_zh = monolingual_learning_prod(dat_zh, cue_obj_zh, S_zh, 2747, 1);

In [ ]:
plot(prod_en, xlab="evaluation step", ylab="accuracy", label="English")
plot!(prod_de, label="German")
plot!(prod_zh, label="Mandarin")

### Bilingual learning

In [ ]:
dat_bi = vcat(dat_en, dat_de);
cue_obj_bi = JudiLing.make_cue_matrix(dat_bi, grams=3, target_col=:Phon);
S_bi = vcat(S_en, S_de);

#### Simultaneous bilingual: comprehension

In [ ]:
learn_seq_bi = JudiLing.make_learn_seq(dat_bi.simFreq, random_seed=314);
eval_size = 2747
n_evals = Int64(length(learn_seq_bi)/eval_size)

In [ ]:
F = zeros(size(cue_obj_bi.C)[2], size(S_bi)[2])

accs_L1 = [] # English
accs_L2 = [] # German
for i in 1:n_evals
    learn_seq_epoch = learn_seq_bi[(((i-1)*eval_size)+1):i*eval_size]
    F = JudiLing.wh_learn(cue_obj_bi.C, S_bi, learn_seq=learn_seq_epoch, eta=0.01, weights=F)
    Shat = cue_obj_bi.C * F
    acc, R = JudiLing.eval_SC(Shat, S_bi, R=true)
    accs = [R[j[1], j[1]] == R[j] ? 1 : 0 for j in argmax(R, dims = 2)]
    # L1
    acc_L1 = sum(accs[1:nrow(dat_en)])/nrow(dat_en)
    push!(accs_L1, acc_L1)
    # L2
    acc_L2 = sum(accs[(nrow(dat_en)+1):nrow(dat_bi)])/nrow(dat_de)
    push!(accs_L2, acc_L2)
end

In [ ]:
plot(accs_L1, xlab="evaluation step", ylab="accuracy", label="English")
plot!(accs_L2, label="German")

#### Simultaneous bilingual: production

In [ ]:
F = zeros(size(cue_obj_bi.C)[2], size(S_bi)[2])
G = zeros(size(S_bi)[2], size(cue_obj_bi.C)[2])

accs_prod_bi = []
forms_prod_bi = []
for i in 1:n_evals
    learn_seq_epoch = learn_seq_bi[(((i-1)*eval_size)+1):i*eval_size]
    F = JudiLing.wh_learn(cue_obj_bi.C, S_bi, learn_seq=learn_seq_epoch, eta=0.01, weights=F)
    G = JudiLing.wh_learn(S_bi, cue_obj_bi.C, learn_seq=learn_seq_epoch, eta=0.01, weights=G)
    Chat = S_bi * G

    res = JudiLing.learn_paths(dat_bi, cue_obj_bi, S_bi, F, Chat, threshold=0.01, verbose=false)
    # get by-word accuracy and predicted form
     accs_prod = []
     forms_prod = []
     for k in 1:nrow(dat_bi)
         acc_p = JudiLing.iscorrect(cue_obj_bi.gold_ind[k], res[k][1].ngrams_ind)
         push!(accs_prod, acc_p)
         form_p = JudiLing.translate(res[k][1].ngrams_ind, cue_obj_bi.i2f, 3, false, nothing, "#", "")
         push!(forms_prod, form_p)
     end
     push!(accs_prod_bi, accs_prod)
     push!(forms_prod_bi, forms_prod)
end

In [ ]:
# further processing of the production results
accs_prod_L1 = []
accs_prod_L2 = []
intru_in_L1 = []
intru_in_L2 = []

for i in 1:length(forms_prod_bi)
    accs_tmp = accs_prod_bi[i]
    forms_tmp = forms_prod_bi[i]
    # L1
    acc_L1 = sum(accs_tmp[1:nrow(dat_en)])/nrow(dat_en)
    push!(accs_prod_L1, acc_L1)

    p_L1 = forms_tmp[1:nrow(dat_en)]
    intru_L1 = []
    for (j, x) in enumerate(p_L1)
        if x == dat_en.Phon[j]
            push!(intru_L1, false)
        elseif x == dat_de.Phon[j]
            push!(intru_L1, true)
        else
            push!(intru_L1, false)
        end
    end
    push!(intru_in_L1, sum(intru_L1)/length(p_L1))
    
    # L2
    acc_L2 = sum(accs_tmp[(nrow(dat_en)+1):nrow(dat_bi)])/nrow(dat_de)
    push!(accs_prod_L2, acc_L2)

    p_L2 = forms_tmp[(nrow(dat_en)+1):nrow(dat_bi)]
    intru_L2 = []
    for (j, x) in enumerate(p_L2)
        if x == dat_de.Phon[j]
            push!(intru_L2, false)
        elseif x == dat_en.Phon[j]
            push!(intru_L2, true)
        else
            push!(intru_L2, false)
        end
    end
    push!(intru_in_L2, sum(intru_L2)/length(p_L2))

end

In [ ]:
plot(accs_prod_L1, xlab="evaluation step", ylab="accuracy", label="English", legend=:right)
plot!(accs_prod_L2, label="German")
plot!(intru_in_L1, label="G ii E", ls=:dash, lc=1)
plot!(intru_in_L2, label="E ii G", ls=:dash, lc=2)

#### Late bilingual: comprehension

In [ ]:
eval_size = 2747;
L1_only_eval = 5;
bi_eval = 7;
n_rep = 2;
learn_seq_p1 = repeat(JudiLing.make_learn_seq(dat_en.simFreq, random_seed=314), n_rep);
learn_seq_p2 = repeat(JudiLing.make_learn_seq(dat_bi.simFreq, random_seed=314), n_rep);
learn_seq_late = vcat(learn_seq_p1[1:eval_size*L1_only_eval], learn_seq_p2[1:eval_size*bi_eval])
n_evals = Int64(length(learn_seq_late)/eval_size); # 12

In [ ]:
F = zeros(size(cue_obj_bi.C)[2], size(S_bi)[2])
accs_L1 = [] # English
accs_L2 = [] # German
for i in 1:n_evals
    learn_seq_epoch = learn_seq_late[(((i-1)*eval_size)+1):i*eval_size]
    F = JudiLing.wh_learn(cue_obj_bi.C, S_bi, learn_seq=learn_seq_epoch, eta=0.01, weights=F)
    Shat = cue_obj_bi.C * F
    acc, R = JudiLing.eval_SC(Shat, S_bi, R=true)
    accs = [R[j[1], j[1]] == R[j] ? 1 : 0 for j in argmax(R, dims = 2)]
    # L1
    acc_L1 = sum(accs[1:nrow(dat_en)])/nrow(dat_en)
    push!(accs_L1, acc_L1)
    # L2
    acc_L2 = sum(accs[(nrow(dat_en)+1):nrow(dat_bi)])/nrow(dat_de)
    push!(accs_L2, acc_L2)
end

In [ ]:
plot(accs_L1, xlab="evaluation step", ylab="accuracy", label="English")
plot!(accs_L2, label="German")

#### Late bilingual: production

In [ ]:
F = zeros(size(cue_obj_bi.C)[2], size(S_bi)[2])
G = zeros(size(S_bi)[2], size(cue_obj_bi.C)[2])

accs_prod_bi = []
forms_prod_bi = []
for i in 1:n_evals
    learn_seq_epoch = learn_seq_late[(((i-1)*eval_size)+1):i*eval_size]
    F = JudiLing.wh_learn(cue_obj_bi.C, S_bi, learn_seq=learn_seq_epoch, eta=0.01, weights=F)
    G = JudiLing.wh_learn(S_bi, cue_obj_bi.C, learn_seq=learn_seq_epoch, eta=0.01, weights=G)
    Chat = S_bi * G

    res = JudiLing.learn_paths(dat_bi, cue_obj_bi, S_bi, F, Chat, threshold=0.01, verbose=false)
    # get by-word accuracy and predicted form
     accs_prod = []
     forms_prod = []
     for k in 1:nrow(dat_bi)
        if isassigned(res[k], 1)
            acc_p = JudiLing.iscorrect(cue_obj_bi.gold_ind[k], res[k][1].ngrams_ind)
            push!(accs_prod, acc_p)
            form_p = JudiLing.translate(res[k][1].ngrams_ind, cue_obj_bi.i2f, 3, false, nothing, "#", "")
            push!(forms_prod, form_p)
         else
            push!(accs_prod, false)
            push!(forms_prod, "NA")
         end
     end
     push!(accs_prod_bi, accs_prod)
     push!(forms_prod_bi, forms_prod)
end

In [ ]:
# further processing of the production results
accs_prod_L1 = []
accs_prod_L2 = []
intru_in_L1 = []
intru_in_L2 = []

for i in 1:length(forms_prod_bi)
    accs_tmp = accs_prod_bi[i]
    forms_tmp = forms_prod_bi[i]
    # L1
    acc_L1 = sum(accs_tmp[1:nrow(dat_en)])/nrow(dat_en)
    push!(accs_prod_L1, acc_L1)

    p_L1 = forms_tmp[1:nrow(dat_en)]
    intru_L1 = []
    for (j, x) in enumerate(p_L1)
        if x == dat_en.Phon[j]
            push!(intru_L1, false)
        elseif x == dat_de.Phon[j]
            push!(intru_L1, true)
        else
            push!(intru_L1, false)
        end
    end
    push!(intru_in_L1, sum(intru_L1)/length(p_L1))
    
    # L2
    acc_L2 = sum(accs_tmp[(nrow(dat_en)+1):nrow(dat_bi)])/nrow(dat_de)
    push!(accs_prod_L2, acc_L2)

    p_L2 = forms_tmp[(nrow(dat_en)+1):nrow(dat_bi)]
    intru_L2 = []
    for (j, x) in enumerate(p_L2)
        if x == dat_de.Phon[j]
            push!(intru_L2, false)
        elseif x == dat_en.Phon[j]
            push!(intru_L2, true)
        else
            push!(intru_L2, false)
        end
    end
    push!(intru_in_L2, sum(intru_L2)/length(p_L2))

end

In [ ]:
plot(accs_prod_L1, xlab="evaluation step", ylab="accuracy", label="English", legend=:right)
plot!(accs_prod_L2, label="German")
plot!(intru_in_L1, label="G ii E", ls=:dash, lc=1)
plot!(intru_in_L2, label="E ii G", ls=:dash, lc=2)

### Trilingual learning

#### Comprehension

In [ ]:
eval_size = 2747;
L1_only_eval = 5;
bi_eval = 7;
tri_eval = 8;
n_rep = 2;
data_L1 = dat_en;
data_L2 = dat_de;
data_L3 = dat_zh;
S_L1 = S_en;
S_L2 = S_de;
S_L3 = S_zh;

In [ ]:
dat_bi = vcat(data_L1, data_L2);
dat_tri = vcat(data_L1, data_L2, data_L3);
cue_obj_tri = JudiLing.make_cue_matrix(dat_tri, grams=3, target_col=:Phon);
S_tri = vcat(S_en, S_de, S_zh);

In [ ]:
learn_seq_p1 = repeat(JudiLing.make_learn_seq(data_L1.simFreq, random_seed=314), n_rep);
learn_seq_p2 = repeat(JudiLing.make_learn_seq(dat_bi.simFreq, random_seed=314), n_rep);
learn_seq_p3 = repeat(JudiLing.make_learn_seq(dat_tri.simFreq, random_seed=314), n_rep);
learn_seq = vcat(learn_seq_p1[1:eval_size*L1_only_eval], learn_seq_p2[1:eval_size*bi_eval], learn_seq_p3[1:eval_size*tri_eval])
n_evals = Int64(length(learn_seq)/eval_size) #20

In [ ]:
F = zeros(size(cue_obj_tri.C)[2], size(S_tri)[2])

accs_L1 = []
accs_L2 = []
accs_L3 = []
for i in 1:n_evals
    learn_seq_epoch = learn_seq[(((i-1)*eval_size)+1):i*eval_size]
    F = JudiLing.wh_learn(cue_obj_tri.C, S_tri, learn_seq=learn_seq_epoch, eta=0.01, weights=F, n_epochs=1)
    Shat = cue_obj_tri.C * F
    # strict evaluation
    acc, R = JudiLing.eval_SC(Shat, S_tri, R=true)
    accs = [R[j[1], j[1]] == R[j] ? 1 : 0 for j in argmax(R, dims = 2)]

    # L1
    acc_L1 = sum(accs[1:nrow(data_L1)])/nrow(data_L1)
    push!(accs_L1, acc_L1)
    # L2
    acc_L2 = sum(accs[(nrow(data_L1)+1):nrow(dat_bi)])/nrow(data_L2)
    push!(accs_L2, acc_L2)
    # L3
    acc_L3 = sum(accs[(nrow(dat_bi)+1):nrow(dat_tri)])/nrow(data_L3)
    push!(accs_L3, acc_L3)
end

In [ ]:
plot(accs_L1, xlab="evaluation step", ylab="accuracy", label="English")
plot!(accs_L2, label="German")
plot!(accs_L3, label="Mandarin")

#### Production

In [ ]:
F = zeros(size(cue_obj_tri.C)[2], size(S_tri)[2])
G = zeros(size(S_tri)[2], size(cue_obj_tri.C)[2])

accs_prod_tri = []
forms_prod_tri = []
for i in 1:n_evals
    learn_seq_epoch = learn_seq[(((i-1)*eval_size)+1):i*eval_size]
    F = JudiLing.wh_learn(cue_obj_tri.C, S_tri, learn_seq=learn_seq_epoch, eta=0.01, weights=F, n_epochs=1)
    G = JudiLing.wh_learn(S_tri, cue_obj_tri.C, learn_seq=learn_seq_epoch, eta=0.01, weights=G, n_epochs=1)
    Shat = cue_obj_tri.C * F
    Chat = S_tri * G

    # learn path
    res = JudiLing.learn_paths(dat_tri, cue_obj_tri, S_tri, F, Chat, threshold=0.01, verbose=false);
    # get by-word acc and predicted form
     accs_prod = []
     forms_prod = []
     for k in 1:nrow(dat_tri)
         if isassigned(res[k], 1)
            acc_p = JudiLing.iscorrect(cue_obj_tri.gold_ind[k], res[k][1].ngrams_ind)
            push!(accs_prod, acc_p)
            form_p = JudiLing.translate(res[k][1].ngrams_ind, cue_obj_tri.i2f, 3, false, nothing, "#", "")
            push!(forms_prod, form_p)
         else
            push!(accs_prod, false)
            push!(forms_prod, "NA")
         end
     end
     push!(accs_prod_tri, accs_prod)
     push!(forms_prod_tri, forms_prod)

end

In [ ]:
accs_prod_L1 = []
accs_prod_L2 = []
accs_prod_L3 = []
intru_in_L1 = []
intru_in_L2 = []
intru_in_L3 = []

for i in 1:length(forms_prod_tri)
    accs_tmp = accs_prod_tri[i]
    forms_tmp = forms_prod_tri[i]
    # L1
    acc_L1 = sum(accs_tmp[1:nrow(data_L1)])/nrow(data_L1)
    push!(accs_prod_L1, acc_L1)

    p_L1 = forms_tmp[1:nrow(data_L1)]
    intru_L1 = []
    for (j, x) in enumerate(p_L1)
        if x == data_L1.Phon[j]
            push!(intru_L1, false)
        elseif x == data_L2.Phon[j] || x == data_L3.Phon[j]
            push!(intru_L1, true)
        else
            push!(intru_L1, false)
        end
    end
    push!(intru_in_L1, sum(intru_L1)/length(p_L1))
    
    # L2
    acc_L2 = sum(accs_tmp[(nrow(data_L1)+1):nrow(dat_bi)])/nrow(data_L2)
    push!(accs_prod_L2, acc_L2)

    p_L2 = forms_tmp[(nrow(data_L1)+1):nrow(dat_bi)]
    intru_L2 = []
    for (j, x) in enumerate(p_L2)
        if x == data_L2.Phon[j]
            push!(intru_L2, false)
        elseif x == data_L1.Phon[j] || x == data_L3.Phon[j]
            push!(intru_L2, true)
        else
            push!(intru_L2, false)
        end
    end
    push!(intru_in_L2, sum(intru_L2)/length(p_L2))

    # L3
    acc_L3 = sum(accs_tmp[(nrow(dat_bi)+1):nrow(dat_tri)])/nrow(data_L3)
    push!(accs_prod_L3, acc_L3)

    p_L3 = forms_tmp[(nrow(dat_bi)+1):nrow(dat_tri)]
    intru_L3 = []
    for (j, x) in enumerate(p_L3)
        if x == data_L3.Phon[j]
            push!(intru_L3, false)
        elseif x == data_L1.Phon[j] || x == data_L2.Phon[j]
            push!(intru_L3, true)
        else
            push!(intru_L3, false)
        end
    end
    push!(intru_in_L3, sum(intru_L3)/length(p_L3))

end

In [ ]:
plot(accs_prod_L1, xlab="evaluation step", ylab="accuracy", label="English", legend=:left)
plot!(accs_prod_L2, label="German")
plot!(accs_prod_L3, label="Mandarin")
plot!(intru_in_L1, label="G/M ii E", ls=:dash, lc=1)
plot!(intru_in_L2, label="E/M ii G", ls=:dash, lc=2)
plot!(intru_in_L3, label="E/G ii M", ls=:dash, lc=3)